In [4]:
import cv2
import dlib
import pyttsx3
import numpy as np
from scipy.spatial import distance as dist
from imutils import face_utils
import time

engine = pyttsx3.init()

# Function to play an audio alert
def play_audio_alert(message):
    engine.say(message)
    engine.runAndWait()

# Function to calculate yawn distance
def cal_yawn(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = dist.euclidean(top_mean, low_mean)
    return distance

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

yawn_thresh = 35
prev_eye_state = "Awake"
prev_yawn_state = "Not Yawning"
ptime = 0

while True:
    _, frame = cap.read()
    gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    ctime = time.time()
    fps = int(1 / (ctime - ptime))
    ptime = ctime
    cv2.putText(frame, f'FPS: {fps}', (frame.shape[1] - 120, frame.shape[0] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 200, 0), 3)

    if cap.get(cv2.CAP_PROP_POS_FRAMES) % 5 == 0:
        faces = face_detector(gray_scale)

    for face in faces:
        face_landmarks = dlib_facelandmark(gray_scale, face)
        leftEye = []
        rightEye = []
        mouth = []

        for n in range(42, 48):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            rightEye.append((x, y))
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)  # Mark right eye landmarks

        for n in range(36, 42):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            leftEye.append((x, y))
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)  # Mark left eye landmarks

        for n in range(48, 68):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            mouth.append((x, y))
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)  # Mark mouth landmarks

        poi_A = dist.euclidean(rightEye[1], rightEye[5])
        poi_B = dist.euclidean(rightEye[2], rightEye[4])
        poi_C = dist.euclidean(rightEye[0], rightEye[3])
        aspect_ratio_right_eye = (poi_A + poi_B) / (2 * poi_C)

        poi_A = dist.euclidean(leftEye[1], leftEye[5])
        poi_B = dist.euclidean(leftEye[2], leftEye[4])
        poi_C = dist.euclidean(leftEye[0], leftEye[3])
        aspect_ratio_left_eye = (poi_A + poi_B) / (2 * poi_C)

        Eye_Rat = (aspect_ratio_left_eye + aspect_ratio_right_eye) / 2
        Eye_Rat = round(Eye_Rat, 2)

        lip_dist = cal_yawn(face_utils.shape_to_np(face_landmarks))

        if Eye_Rat < 0.25 and prev_eye_state != "Drowsy":
            prev_eye_state = "Drowsy"
            play_audio_alert("Drowsiness Detected. Wake up!")

        elif Eye_Rat >= 0.25:
            prev_eye_state = "Awake"

        if lip_dist > yawn_thresh and prev_yawn_state != "Yawning":
            prev_yawn_state = "Yawning"
            play_audio_alert("Yawn Detected. Take a break!")

        elif lip_dist <= yawn_thresh:
            prev_yawn_state = "Not Yawning"

    cv2.putText(frame, prev_eye_state, (50, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    cv2.putText(frame, prev_yawn_state, (50, 150), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 200), 3)

    cv2.imshow("Drowsiness and Yawn Detection", frame)
    key = cv2.waitKey(1)

    if key == 27:
        break
    if key == ord('q') or key == ord('Q'):
        print("Program stopped by user.")
        break

cap.release()
cv2.destroyAllWindows()

Program stopped by user.
